# OxMatch Algorithm 

### 1. Agents: 

#### First cluster (Stable Marriage Problem): Gale-Shapley
1. Proposing: Male looing for Female
2. Proposed: Female looing for Male

#### Second Cluster (Stable Roommate problem): 
3. Male looking for Male

#### Third Cluster  (Stable Roommate problem): :
4. Female looking for Female

#### Two matches for romance:
1. Similar
columns 15-33, except 24, 28. Least distance: larger sum of absolute difference, worse
2. Opposite

### 2. Algorithm

#### 2.1 Difference score: squared difference

Sum of squared difference between any two potential matches

#### 2.2 Deal breakers (column, question):

###### Change pair-wise preference score to +1000

    9. 'What years do you want to match with?'
    37. (38) "There are some political views I don't want to be matched with"
    50. I do not want to match with someone from my college
    51. I am happy to match with with someone from my course
    52. (53) "I don't want a match from..."

### 3. TODO
1. Parse qualitative answers (multiple choice format): Daniel Gagliardi
2. Add all deal breakers: Annie
3. Non-binary groups: Yuan Yuan Sun 
4. MM/FF: Giacomo Rossetti (Yuan Yuan Sun - Annie will give library link)
5. Statistics

In [1]:
# I didn't have matching pre-installed
import sys
!{sys.executable} -m pip install matching

In [2]:
import pandas as pd
import numpy as np
import itertools
from itertools import *

from matching.games import StableRoommates

In [4]:
MT2020 = pd.read_csv('anonymMT2020.csv',index_col=[0])

In [5]:
MT2020.iloc[:,8].value_counts()

Philosophy, Politics and Economics (PPE)     174
Law (Jurisprudence)                          172
History                                      149
English Language and Literature              148
Medicine                                     142
Chemistry                                    130
Engineering Science                          114
Modern Languages                             114
Physics                                      105
Mathematics                                   93
Biology                                       84
Biochemistry (Molecular and Cellular)         75
Geography                                     74
Economics and Management                      61
Classics                                      61
Music                                         41
History and Politics                          39
Modern Languages and Linguistics              32
Biomedical Sciences                           32
Materials Science                             32
Earth Sciences (Geol

In [6]:
#percentage of PPE students represented in OxMatch
174/737

0.23609226594301222

In [7]:
#gender
MT2020.iloc[:,2].value_counts()

Female                 1286
Male                   1025
Prefer not to say         7
Nonbinary                 3
Non-binary                2
Non Binary                2
Non binary                1
Non                       1
attack helicopter         1
nonbinary                 1
I don't                   1
queer                     1
Nonbinary                 1
Non-binary                1
Nonbinary (she/her)       1
Name: I identify as..., dtype: int64

In [8]:
#what gender are people looking for: 
MT2020.iloc[:,4].value_counts()

Male                       1146
Female                      881
Male, Female, Nonbinary     117
Male, Female                 96
Female, Nonbinary            30
Female, Male                 21
Male, Nonbinary              12
Female, Male, Nonbinary      10
Nonbinary                     6
Nonbinary, Female, Male       4
Female, Nonbinary, Male       4
Male, Nonbinary, Female       3
Nonbinary, Female             3
Nonbinary, Male, Female       1
Name: I want to match with people who are, dtype: int64

In [9]:
list(MT2020.columns)

['name',
 'What college are you at?',
 'I identify as...',
 'I want to be matched with people looking for...',
 'I want to match with people who are',
 'How do you feel about an open/polyamorous relationship?',
 'What year are you in?',
 'What years do you want to match with?',
 'What do you study?',
 'Where do you go?',
 'How COVID-Secure is your Date?',
 'What do you talk about?',
 'How far do you go?',
 "It's ok to leave friends on read",
 'Capitalism has failed the planet',
 'University COVID guidelines are too harsh',
 'Sharking is predatory',
 'Drug laws should be loosened',
 'I am scared that if I express my opinions I will be cancelled',
 'Pissing in the sink is normal',
 'How smart are you compared to the average Oxford student?',
 'Was clubbing a large part of your social life?',
 'The Pre-Covid best night out was...',
 'How do you feel about alcohol?',
 'How do you feel about cigarettes?',
 'Is music important in your relationships?',
 'Who are your favourite musicians?',
 '

# 1. Dividing Subsets

In [675]:
# With new paradigm, we no longer distinguish between proposing and proposed

#male looking for female
#proposing = MT2020[(MT2020.iloc[:,2] == 'Male') & (MT2020.iloc[:,4] == 'Female')]
#proposed = MT2020[(MT2020.iloc[:,2] == 'Female') & (MT2020.iloc[:,4] == 'Male')]

In [10]:
MT2020.iloc[:,2].unique()

array(['attack helicopter', 'Male', 'Female', 'Prefer not to say',
       'Nonbinary', 'Non', "I don't", 'Nonbinary ', 'Non-binary',
       'Non Binary', 'nonbinary ', 'queer', 'Nonbinary (she/her)',
       'Non-binary ', 'Non binary '], dtype=object)

In [441]:
# With new paradigm, MM & FF are dealt with like all others but with different dealbreakers

#157 people
#MM = MT2020[(MT2020.iloc[:,4] == 'Male') & (MT2020.iloc[:,6] == 'Male')]

#72 people
#FF = MT2020[(MT2020.iloc[:,4] == 'Female') & (MT2020.iloc[:,6] == 'Female')]

# 2. F looking for F, M looking for M

72 people, randomly divided into 2 groups

In [ ]:
#TODO

# 3. M looking for F & F looking for M 


In [22]:
# We now only take one df and Cartesian product it with itself. Then deal with preferences as dealbreakers
# TODO: implement all negative dealbreakers (can do in some sort of loop through all the relevant columns?)
# TODO: implement positive dealbreaker (crushes)
# Discuss: There may be better ways of dealing with dealbreakers than simply inputting 1e4. There could be an input which excludes values entirely from the stable roommate algorithm e.g. -1 or something. This could become useful for stuff like naughty/nice or just to ensure that we definitely never include some matches (and is more elegant)

def preference_table(df1):
    """
    Take into two dataframes (proposing and proposed), return DataFrame of cartesian product of len(df1) * len(df2)  
    
    """
    
    #helper function: column used to match 
    def prase_df(df):
        parsedDf = df.iloc[:,13:31].drop(columns=df.columns[[22,26]]).fillna(0)
        return parsedDf
     
    parsedDf1 = prase_df(df1)
    #parsedDf2 = prase_df(df2)
    
    n = len(df1) # Should this be parsedDf1?
    matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            
            matrix[i,j] = np.sum((parsedDf1.iloc[i] - parsedDf1.iloc[j])**2) 
            
            #Dealbreaker (Positive): Crushes - NB: one bool column followed by several details columns
            
            #Dealbreaker: Year 
            if (df1.iloc[i,6] not in df1.iloc[j,7]) | (df1.iloc[j,6] not in df1.iloc[i,7]): # Should this be parsedDf1?
                matrix[i,j] += 1e4
            
            #Dealbreaker: Political views - NB: one bool column followed by details column
            #if (df1.iloc[i,8] in df2.iloc[j,9]) | (df2.iloc[j,8] in df1.iloc[i,9]): 
            
            #Dealbreaker: College - NB: two relevant questions for this (don't want my college & don't want specific college bool + details column)
            
            #Dealbreaker: Course - NB: not from my course (need to use course column)
            
            #Dealbreaker: Gender preferences
            
            #Dealbreaker: Do not match with self - This is done later in create_prefs so can leave as 0
                
    
    table = pd.DataFrame(matrix,index = parsedDf1.index,columns = parsedDf1.index)
    return table

def create_prefs(prefrence_table):
    #men_prefs = {}
    #women_prefs = {}
    prefs = {}
    for i in range(prefrence_table.shape[0]):
        prefs['Person '+str(prefrence_table.index[i])] = ['Person ' + str(x) for x in prefrence_table.iloc[i].sort_values().index if i != x]
    
    
    return prefs

In [12]:
MT2020[MT2020.iloc[:,35] == 'Very conservative']

,name,What college are you at?,I identify as...,I want to be matched with people looking for...,I want to match with people who are,How do you feel about an open/polyamorous relationship?,What year are you in?,What years do you want to match with?,What do you study?,Where do you go?,...,Is there a particular college you don't want a match from?,I don't want a match from...,How was the quiz?,Any feedback for next time?,Do you want to share any contact details with your match (other than your Oxford email)?,Submitted At,Token,Unnamed: 60,Unnamed: 61,Unnamed: 62
84,person 2247,Trinity College,Male,NaN,Female,Not interested,Third,"Second, Third, Fourth + / Post Grad",Modern Languages and Linguistics,Walk in the park,...,False,NaN,5,The crush match is a good idea,False,2020-10-17 19:40:49,3mz3dmtnkoqk9jxpc1k663mz3g9zvp7r,NaN,NaN,NaN
177,person 1887,Keble College,Male,NaN,Female,Not interested,Second,"Second, Third, First",Economics and Management,Coffee,...,False,NaN,5,No,False,2020-10-17 21:16:22,t4yr4762iapix5tt4yr47pw7su6i8y06,NaN,NaN,NaN
196,person 1137,St Hugh's College,Male,NaN,Female,Not interested,Third,"Third, Second, First",Biology,Drinks,...,False,NaN,5,NaN,True,2020-10-17 21:44:33,8c352pnmx5rve8c35n8uwgnh48gwdqq4,NaN,NaN,NaN
259,person 2334,The Queen's College,Male,NaN,Female,Not interested,Third,"Second, Third","Philosophy, Politics and Economics (PPE)",Walk in the park,...,True,St Hugh's College,4,Damn you people better have some good data sec...,True,2020-10-18 03:04:29,hvv89c3dqz8m1ds7dphvv89c3mi8a46z,NaN,NaN,NaN
288,person 148,Worcester College,Female,NaN,Male,Happy either-way,Fourth + / Post Grad,"Third, Fourth + / Post Grad",Chemistry,Drinks,...,False,NaN,4,Nope,False,2020-10-18 09:04:10,aligqu9bybn3ot2aligqut6xbabp31t5,NaN,NaN,NaN
315,person 1645,New College,Male,NaN,Male,Happy either-way,First,"First, Second, Third, Fourth + / Post Grad",Physics,Dinner,...,False,NaN,3,No very good,True,2020-10-18 10:43:17,e6n8x60komkxthoyy2e6n8x60mbwuwn6,NaN,NaN,NaN
378,person 2948,New College,Female,NaN,Male,Not interested,Second,"First, Second, Third",Medicine,Dinner,...,False,NaN,4,no,True,2020-10-18 14:12:43,kev8mokexm546frbtv3kev8mo7fhu5j4,NaN,NaN,NaN
447,person 2264,Oriel College,Male,NaN,Female,Must have,Second,"Second, First, Third, Fourth + / Post Grad",Physics,Walk in the park,...,False,NaN,5,Not Particularly,True,2020-10-18 15:38:16,pi67p18ksh5771rtpi67p188fuq6n9xq,NaN,NaN,NaN
583,person 2493,St Edmund Hall,Prefer not to say,Nonbinary,Nonbinary,Must have,First,Fourth + / Post Grad,Biomedical Sciences,Walk in the park,...,False,NaN,5,No,True,2020-10-18 20:15:09,c5qmdcs74f5lkviuou2c5qmnwvvjzfyr,NaN,NaN,NaN
597,person 2796,St Antony's College,Male,NaN,Female,Not interested,Fourth + / Post Grad,"First, Second, Third, Fourth + / Post Grad","Philosophy, Politics and Economics (PPE)",Coffee,...,False,NaN,4,No,True,2020-10-18 20:31:12,m0dcx9veubdm887n5zm0dcxal5kvatnh,NaN,NaN,NaN


In [30]:
testNumber = 2334

In [25]:
MT2020[:testNumber]

,name,What college are you at?,I identify as...,I want to be matched with people looking for...,I want to match with people who are,How do you feel about an open/polyamorous relationship?,What year are you in?,What years do you want to match with?,What do you study?,Where do you go?,...,Is there a particular college you don't want a match from?,I don't want a match from...,How was the quiz?,Any feedback for next time?,Do you want to share any contact details with your match (other than your Oxford email)?,Submitted At,Token,Unnamed: 60,Unnamed: 61,Unnamed: 62
0,person 852,Christ Church,attack helicopter,NaN,Nonbinary,Not interested,Second,"First, Second, Third",History and Politics,Dinner,...,True,Exeter College,4,j,True,2020-10-17 17:19:40,c8kuwkpj23iamn4c8xskjhgp7t8xqmj1,NaN,NaN,NaN
1,person 60,Lady Margaret Hall,Male,NaN,Female,Not interested,Third,"Second, Third, Fourth + / Post Grad",Law (Jurisprudence),Drinks,...,False,NaN,4,Nah,True,2020-10-17 17:59:29,uubh5nxvl9zjpupaosueuubhs2y15tyo,NaN,NaN,NaN
2,person 52,Lady Margaret Hall,Female,NaN,"Male, Female, Nonbinary",Happy either-way,Third,"Second, Third, Fourth + / Post Grad",English Language and Literature,Drinks,...,False,NaN,5,The double negatives in the college and course...,True,2020-10-17 17:59:49,kzujua8vd2y6s8u2onkzujuazu40bfwn,NaN,NaN,"< salient advice here, have updated qs from he..."
3,person 1616,Lincoln College,Female,NaN,Female,Not interested,Second,"Second, Third",Mathematics,Walk in the park,...,False,NaN,3,no,True,2020-10-17 18:53:28,sf7g86qt6vdkr5sf7g8r003r9i00zi2l,NaN,NaN,NaN
4,person 1306,St Hugh's College,Male,NaN,Female,Not interested,Second,"Second, Third, First",History (Ancient and Modern),Coffee,...,False,NaN,3,no,False,2020-10-17 19:05:19,p4e53wo7urugw2r17cp4e53wxt3k9aft,NaN,NaN,NaN
5,person 2465,Christ Church,Male,NaN,Male,Not interested,Second,"First, Second, Third",Modern Languages,Walk in the park,...,False,NaN,5,NaN,False,2020-10-17 19:06:50,rq3zx5a0n4iohi9ef1vrq3zxefbl5699,NaN,NaN,NaN
6,person 1691,Magdalen College,Female,NaN,Male,Not interested,Second,"Second, Third, Fourth + / Post Grad",Medicine,Coffee,...,False,NaN,4,Nah it’s good!,False,2020-10-17 19:07:13,jmgrghnkgzznfdgjmggg5i3y3wk5ax2m,NaN,NaN,NaN
7,person 452,Harris Manchester College,Male,NaN,Female,Not interested,Fourth + / Post Grad,"Third, Fourth + / Post Grad, Second",Oriental Studies,Drinks,...,False,NaN,5,No,True,2020-10-17 19:07:19,w78q1ej3wfkjylfzknv7vw78q1ejf3an,NaN,NaN,NaN
8,person 2116,Exeter College,Female,NaN,"Male, Female, Nonbinary",Not interested,Third,"Third, Fourth + / Post Grad","Philosophy, Politics and Economics (PPE)",Coffee,...,False,NaN,4,no,False,2020-10-17 19:07:58,nljgep36sgyoonlmzcy6sme0iwmf49sp,NaN,NaN,NaN
9,person 1594,Oriel College,Female,NaN,Male,Not interested,Third,"Third, Fourth + / Post Grad, Second",Computer Science,Drinks,...,False,NaN,4,No,False,2020-10-17 19:08:51,j72y38mvgr344lj72742amciaojt5u0y,NaN,NaN,NaN


In [708]:
#proposed[:10]

,name,What college are you at?,I identify as...,I want to be matched with people looking for...,I want to match with people who are,How do you feel about an open/polyamorous relationship?,What year are you in?,What years do you want to match with?,What do you study?,Where do you go?,...,Is there a particular college you don't want a match from?,I don't want a match from...,How was the quiz?,Any feedback for next time?,Do you want to share any contact details with your match (other than your Oxford email)?,Submitted At,Token,Unnamed: 60,Unnamed: 61,Unnamed: 62
6,person 989,Magdalen College,Female,NaN,Male,Not interested,Second,"Second, Third, Fourth + / Post Grad",Medicine,Coffee,...,False,NaN,4,Nah it’s good!,False,2020-10-17 19:07:13,jmgrghnkgzznfdgjmggg5i3y3wk5ax2m,NaN,NaN,NaN
9,person 2628,Oriel College,Female,NaN,Male,Not interested,Third,"Third, Fourth + / Post Grad, Second",Computer Science,Drinks,...,False,NaN,4,No,False,2020-10-17 19:08:51,j72y38mvgr344lj72742amciaojt5u0y,NaN,NaN,NaN
12,person 1306,Magdalen College,Female,NaN,Male,Not interested,Second,"Second, Third, Fourth + / Post Grad",History,Drinks,...,False,NaN,5,💕✨💕✨,True,2020-10-17 19:09:58,aj9bfuo3qdy2fj3daj9bf06b07n89eqq,NaN,NaN,NaN
13,person 2351,Exeter College,Female,NaN,Male,Not interested,Third,"Third, Second, Fourth + / Post Grad",Medicine,Drinks,...,False,NaN,5,No,False,2020-10-17 19:10:19,w7abhhahj6bigw7abh0evxq1etrfgl5z,NaN,NaN,NaN
14,person 1381,Somerville College,Female,NaN,Male,Not interested,Second,"Third, Fourth + / Post Grad, Second","Philosophy, Politics and Economics (PPE)",Drinks,...,False,NaN,4,Excellent stuff,True,2020-10-17 19:10:35,weurx7elr2hweurx7f6do3yqz6v97l2y,NaN,NaN,NaN
18,person 758,Wadham College,Female,NaN,Male,Not interested,Fourth + / Post Grad,"Fourth + / Post Grad, Third",Law (Jurisprudence),Drinks,...,False,NaN,5,Nope,True,2020-10-17 19:11:17,5m8gik8tda7mlft5m8gikw984c8jhvgz,NaN,NaN,NaN
20,person 1730,Lady Margaret Hall,Female,NaN,Male,Not interested,Second,"Second, First",Biochemistry (Molecular and Cellular),Walk in the park,...,False,NaN,5,N/a,True,2020-10-17 19:11:25,tlvcdv3r2ebqrfs8d4rtlvcdvkvu7j1g,NaN,NaN,NaN
21,person 2546,Exeter College,Female,NaN,Male,Not interested,Third,"Second, Third, Fourth + / Post Grad",Medicine,Drinks,...,False,NaN,5,No,False,2020-10-17 19:11:44,woiqdslzs3e060fq7itwoiqdsz41kcb4,NaN,NaN,NaN
22,person 780,Magdalen College,Female,NaN,Male,Not interested,Third,"Second, Third, Fourth + / Post Grad",Modern Languages,Drinks,...,False,NaN,3,nothing,True,2020-10-17 19:12:17,k2y0whsqjmtffth7dck2y0kbag0nrpxk,NaN,NaN,NaN
23,person 2571,Trinity College,Female,NaN,Male,Not interested,Second,"Second, Third",Medicine,Walk in the park,...,False,NaN,4,Ask more questions about sport,True,2020-10-17 19:12:37,7n5nq4syn96n3yc7n5nq45j0j9rce23i,NaN,NaN,NaN


In [26]:
preference_table(MT2020[:testNumber])

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.0,70.0,79.0,42.0,23.0,23.0,48.0,10052.0,10035.0,24.0,...,55.0,51.0,59.0,41.0,71.0,25.0,48.0,42.0,63.0,55.0
1,70.0,0.0,17.0,46.0,75.0,55.0,44.0,42.0,39.0,28.0,...,25.0,27.0,33.0,53.0,23.0,10035.0,32.0,28.0,45.0,49.0
2,79.0,17.0,0.0,53.0,76.0,64.0,57.0,35.0,44.0,45.0,...,20.0,14.0,28.0,58.0,14.0,10028.0,33.0,39.0,24.0,50.0
3,42.0,46.0,53.0,0.0,41.0,29.0,30.0,10060.0,10017.0,30.0,...,29.0,51.0,37.0,55.0,45.0,25.0,18.0,34.0,45.0,49.0
4,23.0,75.0,76.0,41.0,0.0,16.0,53.0,10049.0,10038.0,19.0,...,60.0,58.0,56.0,50.0,76.0,28.0,47.0,49.0,60.0,34.0
5,23.0,55.0,64.0,29.0,16.0,0.0,45.0,10043.0,10032.0,13.0,...,44.0,42.0,34.0,26.0,62.0,16.0,41.0,23.0,52.0,40.0
6,48.0,44.0,57.0,30.0,53.0,45.0,0.0,76.0,10017.0,32.0,...,23.0,61.0,29.0,59.0,43.0,31.0,32.0,38.0,41.0,71.0
7,10052.0,42.0,35.0,10060.0,10049.0,10043.0,76.0,0.0,53.0,34.0,...,33.0,23.0,39.0,45.0,27.0,10029.0,38.0,26.0,10031.0,35.0
8,10035.0,39.0,44.0,10017.0,10038.0,10032.0,10017.0,53.0,0.0,27.0,...,18.0,46.0,18.0,10060.0,36.0,10026.0,10021.0,35.0,10042.0,42.0
9,24.0,28.0,45.0,30.0,19.0,13.0,32.0,34.0,27.0,0.0,...,35.0,35.0,31.0,25.0,43.0,10017.0,36.0,20.0,45.0,31.0


In [28]:
create_prefs(preference_table(MT2020[:testNumber]))

{'Person 0': ['Person 4',
  'Person 5',
  'Person 16',
  'Person 28',
  'Person 10',
  'Person 9',
  'Person 36',
  'Person 45',
  'Person 31',
  'Person 39',
  'Person 22',
  'Person 43',
  'Person 15',
  'Person 3',
  'Person 47',
  'Person 26',
  'Person 37',
  'Person 6',
  'Person 46',
  'Person 25',
  'Person 20',
  'Person 33',
  'Person 41',
  'Person 14',
  'Person 40',
  'Person 35',
  'Person 49',
  'Person 30',
  'Person 42',
  'Person 48',
  'Person 11',
  'Person 27',
  'Person 32',
  'Person 13',
  'Person 1',
  'Person 44',
  'Person 12',
  'Person 21',
  'Person 38',
  'Person 23',
  'Person 2',
  'Person 17',
  'Person 24',
  'Person 19',
  'Person 8',
  'Person 7',
  'Person 18',
  'Person 34',
  'Person 29'],
 'Person 1': ['Person 2',
  'Person 44',
  'Person 40',
  'Person 18',
  'Person 34',
  'Person 38',
  'Person 21',
  'Person 41',
  'Person 47',
  'Person 27',
  'Person 9',
  'Person 24',
  'Person 35',
  'Person 46',
  'Person 23',
  'Person 42',
  'Person 1

In [712]:
#create_prefs(preference_table(proposing[:10],proposed[:10]))[0]

{'Man 1': ['Woman 18',
  'Woman 21',
  'Woman 9',
  'Woman 23',
  'Woman 14',
  'Woman 13',
  'Woman 12',
  'Woman 22',
  'Woman 6',
  'Woman 20'],
 'Man 4': ['Woman 9',
  'Woman 20',
  'Woman 14',
  'Woman 22',
  'Woman 6',
  'Woman 13',
  'Woman 23',
  'Woman 21',
  'Woman 12',
  'Woman 18'],
 'Man 7': ['Woman 12',
  'Woman 18',
  'Woman 9',
  'Woman 21',
  'Woman 13',
  'Woman 14',
  'Woman 22',
  'Woman 6',
  'Woman 20',
  'Woman 23'],
 'Man 10': ['Woman 20',
  'Woman 6',
  'Woman 9',
  'Woman 12',
  'Woman 13',
  'Woman 14',
  'Woman 18',
  'Woman 21',
  'Woman 22',
  'Woman 23'],
 'Man 15': ['Woman 9',
  'Woman 14',
  'Woman 20',
  'Woman 22',
  'Woman 6',
  'Woman 13',
  'Woman 21',
  'Woman 23',
  'Woman 12',
  'Woman 18'],
 'Man 28': ['Woman 9',
  'Woman 18',
  'Woman 21',
  'Woman 13',
  'Woman 14',
  'Woman 22',
  'Woman 6',
  'Woman 23',
  'Woman 12',
  'Woman 20'],
 'Man 29': ['Woman 18',
  'Woman 6',
  'Woman 9',
  'Woman 12',
  'Woman 14',
  'Woman 22',
  'Woman 21',
  '

In [ ]:
#Creating matches
#men_prefs = create_prefs(preference_table(proposing[:10],proposed[:10]))[0]
#women_prefs = create_prefs(preference_table(proposing[:10],proposed[:10]))[1]
prefs = create_prefs(preference_table(MT2020[:testNumber]))

game = StableRoommates.create_from_dictionary(prefs)
result = game.solve()
pd.DataFrame(result.items()).astype(str)

In [358]:
# TODO: Since we're no longer explicitly dealing with gender, we need to find a way to deal with extra women - they will currently just be matched with other women. In theory, if there are extra women, all the bisexual women should be matched with other women to reduce the excess but there may still be an excess that we need to deal with.
# Proposed solution: Remove the excess girls in advance and then repeat the matrix with them and randomly-selected guys
# Alternative: Run the single matrix and then retrieve all the worst matches and deal with them separately

preference_table = pd.DataFrame(a,index = parsedProposing.index,columns = parsedProposed.index)

#randomly drop the extra women
remove = np.random.choice(np.arange(988),size=181,replace=False)
dropped_preference_table = preference_table.drop(preference_table.columns[remove],axis=1)
dropped_preference_table

,6,9,12,13,14,18,20,21,22,23,...,2311,2312,2318,2319,2320,2323,2329,2330,2331,2332
1,18.0,18.0,20.0,15.0,17.0,15.0,24.0,13.0,18.0,19.0,...,21.0,15.0,12.0,20.0,23.0,20.0,18.0,21.0,18.0,19.0
4,23.0,13.0,33.0,28.0,20.0,26.0,19.0,30.0,21.0,28.0,...,28.0,24.0,29.0,13.0,18.0,17.0,21.0,20.0,23.0,36.0
7,26.0,18.0,16.0,21.0,19.0,17.0,26.0,19.0,20.0,21.0,...,21.0,17.0,22.0,24.0,25.0,22.0,18.0,25.0,20.0,21.0
10,20.0,14.0,24.0,23.0,19.0,23.0,16.0,21.0,20.0,21.0,...,17.0,19.0,26.0,12.0,13.0,12.0,16.0,21.0,14.0,23.0
15,20.0,12.0,26.0,19.0,13.0,17.0,16.0,21.0,16.0,23.0,...,25.0,15.0,20.0,18.0,21.0,18.0,14.0,25.0,16.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316,16.0,12.0,26.0,21.0,13.0,15.0,18.0,21.0,16.0,23.0,...,19.0,19.0,20.0,12.0,15.0,14.0,16.0,19.0,14.0,21.0
2324,31.0,21.0,35.0,22.0,30.0,28.0,21.0,28.0,27.0,24.0,...,32.0,26.0,31.0,15.0,20.0,23.0,27.0,26.0,27.0,36.0
2325,20.0,22.0,14.0,17.0,23.0,17.0,26.0,11.0,18.0,19.0,...,15.0,15.0,16.0,20.0,25.0,20.0,16.0,25.0,18.0,13.0
2327,21.0,15.0,23.0,16.0,16.0,16.0,13.0,14.0,17.0,18.0,...,22.0,10.0,17.0,19.0,22.0,21.0,11.0,24.0,13.0,24.0


In [386]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(10000)

5000


In [388]:
from matching.games import StableMarriage
game = StableMarriage.create_from_dictionaries(hospital_prefs, resident_prefs)
result = game.solve()